<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/FeatureGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install textblob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

# Reading the Track Information Details

In [5]:
track_df = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/track_data/tracks_info_0-9999.csv")

track_df

#should be 34443 tracks, not 34442

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",83,0.664,0.758,2,-6.583,0,...,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,Diamond Child,Jon D,spotify:artist:1r2kTJ27zuaEoXasQT5NDd,['channel pop'],39,0.669,0.228,2,-12.119,1,...,0.0944,0.402,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4
34438,I Don't Know,Big Words,spotify:artist:5HCypjplgh5uQezvBpOfXN,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,0.1290,0.289,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4
34439,The Answer,Allan Rayman,spotify:artist:0sHN89qak07mnug3LVVjzP,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,0.2980,0.265,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4
34440,25.22,Jon Jason,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,[],2,0.509,0.286,8,-14.722,1,...,0.1310,0.259,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4


# Analysis of Track Name using Sentiment Analysis
Subjectivity [0:1]: 0 meaning factual, whereas 1 means opinionated.
Polarity [-1,1]: -1 is negative connotation, 1 means positive connotation

In [6]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  # Perform sentiment analysis on text
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [7]:
# Show result
sentiment = sentiment_analysis(track_df, "track_name")
sentiment

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,subjectivity,polarity
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,low,Neutral
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,low,Neutral
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",83,0.664,0.758,2,-6.583,0,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,high,Negative
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,low,Neutral
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,low,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,Diamond Child,Jon D,spotify:artist:1r2kTJ27zuaEoXasQT5NDd,['channel pop'],39,0.669,0.228,2,-12.119,1,...,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4,low,Neutral
34438,I Don't Know,Big Words,spotify:artist:5HCypjplgh5uQezvBpOfXN,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4,low,Neutral
34439,The Answer,Allan Rayman,spotify:artist:0sHN89qak07mnug3LVVjzP,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4,low,Neutral
34440,25.22,Jon Jason,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,[],2,0.509,0.286,8,-14.722,1,...,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4,low,Neutral


In [8]:
def ohe_process(df, column, new_name):
  tf_df = pd.get_dummies(df[column])
  feature_names = tf_df.columns
  tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
  tf_df.reset_index(drop = True, inplace = True)
  return tf_df

In [9]:
def subjectivity(df):
  subject_ohe = ohe_process(df, "subjectivity", "subjectivity") * 0.3
  return subject_ohe

In [10]:
def polarity(df):
  polarity_ohe = ohe_process(df, "polarity", "polarity") * 0.5
  return polarity_ohe

# Analysis on Genres
Use Term Frequency-Inverse Document Frequency to determine the importance of each genre.

In [11]:
def genre_tfidf(df):
  tfidf = TfidfVectorizer()
  tfidf_track_matrix = tfidf.fit_transform(df['genres'])
  genre_df = pd.DataFrame(tfidf_track_matrix.toarray())
  genre_df.columns = ['genre' + '|' + i for i in tfidf.get_feature_names_out()]
  return genre_df

  # genre_df['genre|rap']

# Normalization on Floating Values
Analysis on popularity, key, mode, and other audio features

In [12]:
def popularity_normalization(df):
  popularity = df[['popularity']]
  scaler = MinMaxScaler()
  popularity_df = pd.DataFrame(scaler.fit_transform(popularity), columns = ['popularity'])
  return popularity_df

In [13]:
# audio features
def audio_features(df):
  #one-hot encoding
  key_ohe = ohe_process(df, "key", "key") * 0.5
  mode_ohe = ohe_process(df, "mode", "mode") * 0.5
  #normalization of other audio_features
  floats_cols = df.dtypes[df.dtypes == 'float64'].index.values #selects the audio feature colns
  floats = df[floats_cols].reset_index(drop = True)
  scaler = MinMaxScaler()
  floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
  #putting them altogether
  audio_feats_df = pd.concat([key_ohe, mode_ohe, floats_scaled], axis=1)
  return audio_feats_df

audio_features(track_df)


,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34438,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930


# Generating the Feature Set

In [14]:
def generate_feature_set(df):
  popularity = popularity_normalization(df)
  genre = genre_tfidf(df)
  subjectivity_names = subjectivity(df)
  audio_features_df = audio_features(df)

  feature_set = pd.concat([genre, popularity, subjectivity_names, audio_features_df], axis = 1)
  return feature_set

generate_feature_set(track_df)

,genre|21st,genre|432hz,genre|abstract,genre|acid,genre|acousmatic,genre|acoustic,genre|action,genre|adoracion,genre|adult,genre|adventista,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930


# Picking a playlist

In [15]:
# generating playlist recommendations
playlist_df = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/track_data/playlist_info_0-9999.csv")
playlist_df.head()

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,playlist_pid,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,0,Throwbacks,False,1493424000,52,47,1,0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863
1,0,Throwbacks,False,1493424000,52,47,1,0,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800
2,0,Throwbacks,False,1493424000,52,47,1,0,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933
3,0,Throwbacks,False,1493424000,52,47,1,0,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266
4,0,Throwbacks,False,1493424000,52,47,1,0,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600


# Processing playlists

In [18]:
#feature set of all tracks
total_tracks_features = generate_feature_set(track_df)
total_tracks_features['track_uri'] = track_df['uri']
total_tracks_features

# feature set summary vector for a playlist
def input_playlist_features_vector (pid):
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])

  input_playlist_features_df = total_tracks_features[total_tracks_features['track_uri'].isin(playlist_uris)]
  input_playlist_features_df = input_playlist_features_df.drop('track_uri', axis = 1)
  feature_vector = input_playlist_features_df.sum(axis=0)
  return feature_vector

input_playlist_features_vector(0)

<ipython-input-18-9c7dc4c1daa3>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])


genre|21st          0.000000
genre|432hz         0.000000
genre|abstract      0.000000
genre|acid          0.000000
genre|acousmatic    0.000000
                      ...   
acousticness        0.916746
instrumentalness    0.007142
liveness            1.925840
valence             6.707415
tempo               5.708442
Length: 1357, dtype: float64

In [19]:
def generate_nonplaylist_features(pid):
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])
  nonplaylist_features = total_tracks_features[~total_tracks_features['track_uri'].isin(playlist_uris)]
  nonplaylist_features = nonplaylist_features.drop('track_uri', axis = 1)
  nonplaylist_features.reset_index(drop = True, inplace = True)
  return nonplaylist_features

generate_nonplaylist_features(0)

<ipython-input-19-490ff16ca0d4>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])


,genre|21st,genre|432hz,genre|abstract,genre|acid,genre|acousmatic,genre|acoustic,genre|action,genre|adoracion,genre|adult,genre|adventista,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.069433,0.1950,0.183214,0.028067,0.000006,0.199196,0.01030,0.026653,0.109334
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.083806,0.1918,0.177488,0.007859,0.000005,0.187538,0.07360,0.118036,0.133072
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.105668,0.0410,0.165739,0.007817,0.010723,0.047236,0.01970,0.070741,0.094728
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.017186,0.1912,0.177816,0.016736,0.000011,0.191558,0.07940,0.061924,0.169083
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.057287,0.1850,0.176210,0.023077,0.000150,0.187538,0.03320,0.073747,0.090699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930


# Implementing Recommendation System

In [31]:
def recommend_playlist(input_pid):
  input_playlist_features = input_playlist_features_vector(input_pid)
  nonplaylist_features = generate_nonplaylist_features(input_pid)

  playlist_uris = list(track_df[playlist_df['pid']==input_pid]['uri'])
  nonplaylist_df = track_df[~track_df['uri'].isin(playlist_uris)]
  nonplaylist_df.reset_index(drop = True, inplace = True)
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]

  nonplaylist_df_top_40 = nonplaylist_df.sort_values('similarity',ascending = False).head(40)
  return nonplaylist_df_top_40

recommend_playlist(0)

<ipython-input-18-9c7dc4c1daa3>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])
<ipython-input-19-490ff16ca0d4>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  playlist_uris = list(track_df[playlist_df['pid']==pid]['uri'])
<ipython-input-31-883155735b0e>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  playlist_uris = list(track_df[playlist_df['pid']==input_pid]['uri'])
<ipython-input-31-883155735b0e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]


,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,subjectivity,polarity,similarity
15405,Another World,Black Eyed Peas,spotify:artist:7gAppWoH7pcYmphCVTXkzs,"['dance pop', 'pop', 'pop rap']",77,0.785,0.794,2,-2.197,1,...,audio_features,3qK8x4GZcIkzTz9JEqvIF5,spotify:track:3qK8x4GZcIkzTz9JEqvIF5,https://api.spotify.com/v1/tracks/3qK8x4GZcIkz...,https://api.spotify.com/v1/audio-analysis/3qK8...,217213,4,low,Neutral,0.877782
12135,All We Need,Taylor Swift,spotify:artist:21mKp7DqtSNHhCAU2ugvUw,['pop'],100,0.668,0.672,2,-4.931,1,...,audio_features,15DeqWWQB4dcEWzJg15VrN,spotify:track:15DeqWWQB4dcEWzJg15VrN,https://api.spotify.com/v1/tracks/15DeqWWQB4dc...,https://api.spotify.com/v1/audio-analysis/15De...,201107,4,low,Neutral,0.876651
15576,Iscariot,Taylor Swift,spotify:artist:6DIS6PRrLS3wbnZsf7vYic,['pop'],100,0.815,0.377,4,-7.754,1,...,audio_features,1NmVZsG18CzCAtw7rnV3yA,spotify:track:1NmVZsG18CzCAtw7rnV3yA,https://api.spotify.com/v1/tracks/1NmVZsG18CzC...,https://api.spotify.com/v1/audio-analysis/1NmV...,271000,4,low,Neutral,0.876268
14935,Dive In,Taylor Swift,spotify:artist:7hu7hjmIPOtU1S3D8zfBlw,['pop'],100,0.633,0.889,5,-5.870,1,...,audio_features,0qAIiGFKLdV1xpNlEhjpq8,spotify:track:0qAIiGFKLdV1xpNlEhjpq8,https://api.spotify.com/v1/tracks/0qAIiGFKLdV1...,https://api.spotify.com/v1/audio-analysis/0qAI...,230467,4,low,Neutral,0.875947
2391,Crazier,Taylor Swift,spotify:artist:06HL4z0CvFAxyc27GXpf02,['pop'],100,0.452,0.544,4,-5.423,1,...,audio_features,5vyxXfD5gLlyPxGZMEjtmd,spotify:track:5vyxXfD5gLlyPxGZMEjtmd,https://api.spotify.com/v1/tracks/5vyxXfD5gLly...,https://api.spotify.com/v1/audio-analysis/5vyx...,191947,3,low,Neutral,0.875945
31492,Lo Estoy Intentando,Taylor Swift,spotify:artist:66ArjpKRgw8vYBf9yhktto,['pop'],100,0.618,0.741,2,-3.950,1,...,audio_features,1vrd6UOGamcKNGnSHJQlSt,spotify:track:1vrd6UOGamcKNGnSHJQlSt,https://api.spotify.com/v1/tracks/1vrd6UOGamcK...,https://api.spotify.com/v1/audio-analysis/1vrd...,235267,4,low,Neutral,0.875215
15582,Plage,Taylor Swift,spotify:artist:75EZuo5MHV2572NRpMWotC,['pop'],100,0.705,0.621,2,-8.086,1,...,audio_features,5kYFVSQoPu7yRpfiHBwMUk,spotify:track:5kYFVSQoPu7yRpfiHBwMUk,https://api.spotify.com/v1/tracks/5kYFVSQoPu7y...,https://api.spotify.com/v1/audio-analysis/5kYF...,240773,4,low,Neutral,0.875118
21713,It's Alright,Bruno Mars,spotify:artist:7sjRnhONmeFL1tmlUvdq70,"['dance pop', 'pop']",86,0.692,0.896,1,-4.017,1,...,audio_features,1KtU0WCq472KzqCXgMOxkS,spotify:track:1KtU0WCq472KzqCXgMOxkS,https://api.spotify.com/v1/tracks/1KtU0WCq472K...,https://api.spotify.com/v1/audio-analysis/1KtU...,195889,4,low,Neutral,0.874910
32880,Souvenir,Taylor Swift,spotify:artist:4ClsVDy2g7RKSSlvq8cF6d,['pop'],100,0.436,0.950,5,-2.627,1,...,audio_features,28M2gifMU282QBM3fKajIS,spotify:track:28M2gifMU282QBM3fKajIS,https://api.spotify.com/v1/tracks/28M2gifMU282...,https://api.spotify.com/v1/audio-analysis/28M2...,242080,4,low,Neutral,0.874085
16898,Somebody Else,Bruno Mars,spotify:artist:20s0P9QLxGqKuCsGwFsp7w,"['dance pop', 'pop']",86,0.854,0.564,1,-4.964,1,...,audio_features,1BEMASUZQDqN5UGxMb1m6A,spotify:track:1BEMASUZQDqN5UGxMb1m6A,https://api.spotify.com/v1/tracks/1BEMASUZQDqN...,https://api.spotify.com/v1/audio-analysis/1BEM...,234596,4,low,Neutral,0.872980


# Things to do next
1. Generate the accuracy of the recommended playlist
2. Expand to entire dataset.
3. Implement an input-playlist function.
4. Front end?